In [ ]:
import numpy as np
import cupy as cp
from holotomocupy.holo import G, GT
from holotomocupy.magnification import M, MT
from holotomocupy.shift import S, ST
from holotomocupy.recon_methods import multiPaganin
from holotomocupy.utils import *
from holotomocupy.proc import remove_outliers
import holotomocupy.chunking as chunking
import sys

chunk = 5
chunking.global_chunk = chunk


# Init data sizes and parametes of the PXM of ID16A

In [ ]:
n = 2048  # object size in each dimension
pad = 0#n//8
ndist = 4
lam = 0.1
show = True
# ntheta = int(sys.argv[1])
# st = int(sys.argv[2])
# gpu = int(sys.argv[3])
ntheta=150
st = 0

flg = f'{n}_{ntheta}_{pad}_{lam}_{st}'

detector_pixelsize = 3.03751e-6
energy = 33.35  # [keV] xray energy
wavelength = 1.2398419840550367e-09/energy  # [m] wave length
focusToDetectorDistance = 1.28  # [m]
sx0 = 1.286e-3
z1 = np.array([4.236e-3,4.3625e-3,4.86850e-3,5.91950e-3])[:ndist]-sx0
z2 = focusToDetectorDistance-z1
distances = (z1*z2)/focusToDetectorDistance
magnifications = focusToDetectorDistance/z1
voxelsize = detector_pixelsize/magnifications[0]*2048/n  # object voxel size

norm_magnifications = magnifications/magnifications[0]
# scaled propagation distances due to magnified probes
distances = distances*norm_magnifications**2

z1p = z1[0]  # positions of the probe for reconstruction
z2p = z1-np.tile(z1p, len(z1))
# magnification when propagating from the probe plane to the detector
magnifications2 = (z1p+z2p)/z1p
# propagation distances after switching from the point source wave to plane wave,
distances2 = (z1p*z2p)/(z1p+z2p)
norm_magnifications2 = magnifications2/(z1p/z1[0])  # normalized magnifications
# scaled propagation distances due to magnified probes
distances2 = distances2*norm_magnifications2**2
distances2 = distances2*(z1p/z1)**2

# sample size after demagnification
ne = int(np.ceil((n+2*pad)/norm_magnifications[-1]/32))*32  # make multiple of 32


path = f'/data/vnikitin/ESRF/ID16A/20240924/AtomiumS2/'
pfile = f'AtomiumS2_HT_007nm'
path_out = f'/data/vnikitin/ESRF/ID16A/20240924_rec/AtomiumS2/{pfile}/{flg}'
print(f'{voxelsize=}')
n0=n
ne0=ne
pad0=pad
voxelsize0=voxelsize
print(norm_magnifications)

## Read data

In [ ]:
shifts_init = np.load('shifts.npy')
shifts = shifts_init.copy()
iter =128
for st in range(0,1800,150):
    flg = f'{n}_{ntheta}_{pad}_{lam}_{st}'
    shifts[st:st+150]=2*np.load(f'/data/vnikitin/ESRF/ID16A/20240924_rec/AtomiumS2/{pfile}/{flg}/crec_shift{flg}_{iter:03}.npy')

fig, axs = plt.subplots(1, 2, figsize=(9, 3))
for k in range(4):        
    axs[0].plot(shifts_init[:,k,0]-shifts[:,k,0],'.',label=f"{k}")
    axs[1].plot(shifts_init[:,k,1]-shifts[:,k,1],'.',label=f"{k}")
    plt.legend()
plt.show()
for st in range(0,1800,150):
    print(st)
    flg = f'{n}_{ntheta}_{pad}_{lam}_{st}'
    amp = dxchange.read_tiff(f'/data/vnikitin/ESRF/ID16A/20240924_rec/AtomiumS2/{pfile}/{flg}/crec_psio_abs{flg}/{iter:03}.tiff')
    phase = dxchange.read_tiff(f'/data/vnikitin/ESRF/ID16A/20240924_rec/AtomiumS2/{pfile}/{flg}/crec_psio_angle{flg}/{iter:03}.tiff')
    psi = amp*np.exp(1j*phase)
    mshow_polar(psi,True)



data = np.zeros([1800,ne//2,ne//2],dtype='float32')
prb = np.zeros([1800,4,n//2,n//2],dtype='float32')
for st in range(0,1800,150):
    print(st)
    flg = f'{n}_{ntheta}_{pad}_{lam}_{st}'
    data[st:st+150] = dxchange.read_tiff(f'/data/vnikitin/ESRF/ID16A/20240924_rec/AtomiumS2/{pfile}/{flg}/crec_psi_angle{flg}/{iter:03}.tiff')
    prb[st:st+150] = dxchange.read_tiff(f'/data/vnikitin/ESRF/ID16A/20240924_rec/AtomiumS2/{pfile}/{flg}/crec_prb_angle{flg}/{iter:03}.tiff')

# dxchange.write_tiff_stack(data,f'/data/vnikitin/ESRF/ID16A/20240924_rec/AtomiumS2/{pfile}/ddata',overwrite=True)
# for k in range(4):
#     dxchange.write_tiff_stack(prb[:,k],f'/data/vnikitin/ESRF/ID16A/20240924_rec/AtomiumS2/{pfile}/pprb{k}',overwrite=True)
# dxchange.write_tiff(np.angle(vars['psi'])[0],f'{path_out}/crec_psio_angle{flg}/{i:03}',overwrite=True)
# dxchange.write_tiff(np.angle(vars['prb'])[0],f'{path_out}/crec_prbo_angle{flg}/{i:03}',overwrite=True)    
# dxchange.write_tiff(np.abs(vars['psi']),f'{path_out}/crec_code_abs{flg}/{i:03}',overwrite=True)
# dxchange.write_tiff(np.abs(vars['prb']),f'{path_out}/crec_prb_abs{flg}/{i:03}',overwrite=True)
# dxchange.write_tiff(np.abs(vars['psi'])[0],f'{path_out}/crec_psio_abs{flg}/{i:03}',overwrite=True)
# dxchange.write_tiff(np.abs(vars['prb'])[0],f'{path_out}/crec_prbo_abs{flg}/{i:03}',overwrite=True)
# np.save(f'{path_out}/crec_shift{flg}_{i:03}',vars['shift'])
# print(f'saved to {path_out}/crec_psi_angle{flg}/{i:03}',flush=True)